In [1]:
import openai
from dotenv import dotenv_values

In [4]:
config = dotenv_values("../../../.env")

In [6]:
openai.api_key = config["openai"]

In [7]:
import textwrap

In [8]:
def nice_print(dictionary):
    for key, value in dictionary.items():
        wrapped_text = "\n".join(textwrap.wrap(value, width=120))
        print(f"{key}:")
        print(wrapped_text)
        print("=" * 20)

## Temperature
- A value from 0-2, though most often between 0 and 1
- It's default is 1
- Controls the randomness of the output. Higher values are more random, lower values are more deterministic

In [11]:
nice_print(
    {
       f"Temperature {temperature}": openai.Completion.create(model="text-davinci-003", prompt="""My favorite food is""".strip(), max_tokens=75, temperature=temperature)\
       .choices[0]["text"] \
       .strip() \
       for temperature in [0, 0.5, 1, 1.5, 2]
    }
)

Temperature 0:
pizza. I love the combination of the doughy crust, the tangy tomato sauce, and the melty cheese. I also like to add
different toppings like pepperoni, mushrooms, and olives. Pizza is a great meal for any occasion, whether it's a casual
night in or a special celebration.
Temperature 0.5:
pizza. I love the combination of cheese, sauce, and a variety of toppings. The crust is usually my favorite part, and I
like to get creative with the toppings. I always get a thin crust pizza because I like the crunchy texture. Pizza is
great for any occasion, from a quick snack to a full meal.
Temperature 1:
chicken nuggets.  Chicken nuggets are crispy on the outside and juicy on the inside. They are also perfect for dipping
in sauces like honey mustard, barbeque, or ranch. They make for a great snack or meal when served with a side of fries
or a salad.
Temperature 1.5:
pizza.
Temperature 2:
watercress dried beef soup. It's a traditional Jingshi soup in Southeast China characterize mainl

---

## Top P
- An alternative to sampling with temperature, called uncleus sampling
- It's default value is 1
- Like temperature, top p alters the "creativity" and randomness of the output
- Top p controls the set of possible words/tokens that the model can choose from
- It restricts the candidate words to the smallest set whose cumulative probability is greater than or equal to a given threshold, "p"

In [13]:
nice_print(
    {
        f"Top p {top_p}": openai.Completion.create(
            model="text-davinci-003",
            prompt="""My favorite food is""".strip(),
            max_tokens=75,
            echo=True,
            top_p=top_p
        )
        .choices[0]["text"]
        .strip()
        for top_p in [1, 0.5, 0]
        
    }
)

Top p 1:
My favorite food is pizza. I love pizza because it's so versatile. You can top it with almost anything and make it
different every time. Whether its loaded with veggies, covered with pepperoni, or smothered in cheese, pizza always hits
the spot.
Top p 0.5:
My favorite food is pizza. I love pizza because it is delicious and there are so many different varieties to choose
from. From classic pepperoni to creative combinations like barbecue chicken and pineapple, there is something for
everyone. Plus, it's easy to make and share with friends and family.
Top p 0:
My favorite food is pizza. I love the combination of the doughy crust, the tangy tomato sauce, and the melty cheese. I
also like to add different toppings like pepperoni, mushrooms, and olives. Pizza is a great meal for any occasion,
whether it's a casual night in or a special celebration.


top p를 늘리고, temperature를 높이면 문서는 하나만 수행할 것을 권장함.


---
## Frequency Penalty
- A number from -2 to 2
- Defaults to 0
- Positive values penalize new tokens based on their existing frequency in the text so far, **decreasing the model's likelihood to repeat the same line verbatim**.
- If you want to reduce repetitive samples, try a penalty from 0.1 - 1
- To strongly suppress repentition, you can increase it further BUT this can lead to bad quality outputs
- Negarive values increase the likelyhood of repetition

In [14]:
nice_print(
{
    f"Frequency Penalty {x}": openai.Completion.create(
        model="text-davinci-003",
        prompt="""The first 15 elements are Hydrogen, Helium,""".strip(),
        max_tokens=200,
        echo=True,
        frequency_penalty=x
    )
    .choices[0]["text"]
    .strip()
    for x in [-2, -1, 0, 1, 2]
})

Frequency Penalty -2:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Frequency Penalty -1:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium,, Aluminum, Silicon, Phosphorus,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Frequency Penalty 0:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminum, Silicon and Phosphorus
Frequency Penalty 1:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnes

---
## Presence Penalty
- A number from -2 to 2
- Defaults to 0
- Positive values penalize new tokens based on whether they appear in the text so far, **increasing the model's likelihood to talk about new topics**.
- Presence panalty is a **one-off additive contiribution** that applies to all tokens that have been sampled at least once
- Frequency penalty is a contribution that is **proportional** to how often a particular token hash already been sampled


In [16]:
nice_print(
{
    f"Presence Penalty {x}": openai.Completion.create(
        model="text-davinci-003",
        prompt="""The first 15 elements are Hydrogen, Helium,""".strip(),
        max_tokens=200,
        echo=True,
        presence_penalty=x
    )
    .choices[0]["text"]
    .strip()
    for x in [-2, -1, 0, 1, 2]
})

Presence Penalty -2:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminum, Silicon, and Phosphorus.
Presence Penalty -1:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminum, Silicon and Phosphorus.
Presence Penalty 0:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminum, Silicon, Phosphorus.
Presence Penalty 1:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminum, Silicon and Phosphorus.
Presence Penalty 2:
The first 15 elements are Hydrogen, Helium, Lithium, Beryllium, Boron, Carbon, Nitrogen, Oxygen, Fluorine, Neon, Sodium,
Magnesium, Aluminium, Silicon, and Phosphorus.


---
## Streaming Responses

In [18]:
openai.Completion.create(
    model="text-davinci-003",
    prompt="write me a poem about foggy morning commutes",
    max_tokens=300
)

<OpenAIObject text_completion id=cmpl-78fqyxrMBXinVjllmh3UQXojCK3Rs at 0x10acd76b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nFog on the streets, a heavy morning chill.\nThe humid air, making me ill.  \nCommuters hurry to catch their bus,\nForgetting to marvel at the morning fog's fuss.\n\nDriving along so it can be seen,\nThe skies are grey and at times mean.\nPassing through the misty street lights,\nA hazy world creates a twilight.\n\nMuffled sounds of car horns blaring.\nThe distant siren of an ambulance caring.\nWrapped in the fog a slow drive ahead,\nSteady and cool, until it\u2019s time to leave our bed."
    }
  ],
  "created": 1682301864,
  "id": "cmpl-78fqyxrMBXinVjllmh3UQXojCK3Rs",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 134,
    "prompt_tokens": 10,
    "total_tokens": 144
  }
}

In [30]:
for data in openai.Completion.create(
    model="text-davinci-003",
    prompt="write me a poem about foggy morning commutes",
    max_tokens=300,
    stream=True
):
    print(data.choices[0].text, end="", flust=True)



When the morning fog sets in, it's a destitute scene
As I drive to work I feel like I'm in a dream
I can't see the road, not even a dim light
Visibility is so poor, it's like I'm in a fight

Driving slowly, I move ahead
The humidity on my glasses, it's like I've been mislead
The limited view that I have, it's a dire sight
Wondering how much longer until morning light

The fog is ever so thick, giving an eerie feeling
But I'm no stranger to this, so there's no need for squealing
As I drove I listened to the radio play a song
Helped me see that I could go the distance and be strong

When I made it to work, fog slowly lifting away
My morning commute had finally come to an end of day